In [5]:
import os
import json

with open("yes_no_left_cnt.txt", "r") as file:
    numbers = file.read().split()
    num_yes_helmet = int(numbers[0])
    num_no_helmet = int(numbers[1])

# 원본 데이터
directory="D:/aihub_data/construction_site_secure_equipment_data/labels/105.공사현장 안전장비 인식 데이터/01.데이터/1.Training/라벨링데이터(zip)_230328_add/고양스타필드/1.안전보호구만"

# 이미지 파일의 S3 경로
# 이 부분은 실제 S3 경로로 수정해야 합니다.
# s3_image_path = "s3://my-bucket/path/to/image.jpg"
s3_image_path = "s3://custom-labels-console-ap-northeast-2-0d6a663a62/assets/ict_makerthon_training_dataset/230530/"

total_count=0
filtered_count=0
valid_filename_list=[]

for filename in os.listdir(directory):
    if filename.endswith('.json'):
        total_count+=1
        is_valid_data=False #안전모가 포함된 데이터 유무
        is_yes_helmet=False
        is_no_helmet=False
        filepath = os.path.join(directory, filename)
        filename_jpg=filename.replace('json','jpg')
        # 파일을 열고 JSON 데이터를 파이썬 dictionary로 읽어들입니다.
        with open(filepath, 'r',encoding='utf-8') as file:
            data = json.load(file)
            
            # 여기서 data는 해당 JSON 파일의 내용을 dictionary 형태로 가지고 있습니다.
            # 이제 이 data를 원하는 대로 처리하면 됩니다
            
            # 변환된 데이터를 저장할 딕셔너리
            transformed_data = {
                "source-ref": s3_image_path+filename_jpg,
                "BB": {
                    "image_size": [
                        {
                            "width": data["image"]["resolution"][0],
                            "height": data["image"]["resolution"][1],
                            "depth": 3  # 보통 이미지는 RGB 3채널입니다.
                        }
                    ],
                    "annotations": []
                },
                "BB-metadata": {
                    "job-name": "labeling-job/BB",
                    "class-map": {
                    "0": "no_helmet",
                    "1": "yes_helmet"
                    },
                    "human-annotated": "yes",
                    "objects": [],
                    "creation-date": "2023-05-30T01:08:59.974Z",
                    "type": "groundtruth/object-detection"
                }
            }
            
            

            # 원본 데이터의 각 어노테이션을 변환된 형태로 추가
            for annotation in data["annotations"]:
                class_id=int(annotation["class"])
                if class_id in [7,8]:
                    if class_id==7 and num_yes_helmet>0:is_yes_helmet=True
                    if class_id==8 and num_no_helmet>0:is_no_helmet=True
                    is_valid_data=True
                    class_id=1 if class_id==7 else 0 #0 -> 8 (미착용) / 1 ->7 (착용)
                    transformed_annotation = {
                        "class_id": class_id,
                        "left": annotation["box"][0],
                        "top": annotation["box"][1],
                        "width": annotation["box"][2] - annotation["box"][0],
                        "height": annotation["box"][3] - annotation["box"][1]
                    }
                    confidence_object={
                            "confidence": 1
                    }
                    transformed_data["BB"]["annotations"].append(transformed_annotation)
                    transformed_data["BB-metadata"]["objects"].append(confidence_object)
            
            # 변환된 데이터를 JSON 문자열로 변환
            transformed_json = json.dumps(transformed_data)
            if is_valid_data and (is_yes_helmet or is_no_helmet):
                filtered_count+=1
                with open('output.manifest','a') as f:
                        f.write(transformed_json+"\n")
                valid_filename_list.append(filename_jpg)
            if is_yes_helmet:num_yes_helmet-=1
            if is_no_helmet:num_no_helmet-=1

for valid_filename in valid_filename_list:
    with open('valid_filename.txt','a') as f:
        f.write(valid_filename+"\n")
        

with open("yes_no_left_cnt.txt", "w") as file:
    file.write(f"{num_yes_helmet} {num_no_helmet}")

{'S3Bucket': 'custom-labels-console-ap-northeast-2-0d6a663a62', 'S3KeyPrefix': 'results/'}
